In [ ]:
!pip install openai

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [16]:
db = SQLDatabase.from_uri("sqlite:///test_prices.db")
llm = ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo')
cadena = SQLDatabaseChain(llm = llm, database = db, verbose=False)
formato = """
            Dada una pregunta del usuario:
            0. Convierte cualquier fecha al formato YYYY-MM-DD para mejor comprension del query
            1. crea una o varias consultas en las tablas de sqlite3 segun la pregunta (no le pongas limit a la consulta), si no se puede crear un query comunica que no tenemos data disponible de esos temas
            2. revisa los resultados, considera todas las columnas importantes al dar la respuesta
            3. devuelve el dato o los datos como un listado (incluye los valores de todas las columnas que consideres relevantes)
            4. añade un resumen con un analisis relevante de los datos mostrados y si hay noticias relacionadas en español
            5. dale una estructura a la respuesta para que sea comoda de leer para el lector
            6. Si no sabes que responder, pregunta al usuario para obtener mas informacion, si no puedes obtener mas informacion, responde con un mensaje de error
            7. Siempre responde en español, y con palabras no tecnicas , un lenguaje cordial
            #{question}
            """

/home/juann/anaconda3/envs/BDI/lib/python3.11/site-packages/langchain_experimental/sql/base.py:76: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [15]:
def consulta(input_usuario):
    consulta = formato.format(question = input_usuario)
    
    try:
        resultado = cadena.run(consulta)
        return resultado
    
    except Exception as e:
        # If there's an exception, prompt ChatGPT for more information
        error_message = f"No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?"
        
        return error_message



In [18]:
#read file line by line
with open('generated_prompts.txt') as f:
    lines = f.readlines()
    for line in lines:
        answer = consulta(line)
        print(answer)
        # write to file
        with open('generated_answers_impact.txt', 'a') as f:
            f.write(answer)
            f.write('\n')

No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?


KeyboardInterrupt: 